# Matplotlib: Event Handling and Picking

# Fontes

+ [Matplotlib Event Handling](https://matplotlib.org/stable/users/event_handling.html)
+ [Get mouse coordinates in Matplotlib](https://stackoverflow.com/questions/51349959/get-mouse-coordinates-without-clicking-in-matplotlib)
+ [Store mouse click event coordinates](https://stackoverflow.com/questions/25521120/store-mouse-click-event-coordinates-with-matplotlib)
+ [Mouse click coordinates as simply as possible](https://stackoverflow.com/questions/37363755/python-mouse-click-coordinates-as-simply-as-possible)

In [1]:
%matplotlib qt5
# %matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np

Exemplo: Mouse Click

In [2]:
fig, ax  = plt.subplots()
ax.plot(np.random.rand(10))
def onclick(event):
    """
    Esta função define o que acontecerá com o plot ao clicar neste. A título de um simples exemplo, o plot
    imprimirá as coordenadas no clique e se o clique foi de botão esquerdo ou direito.
    
    Entrada: 
    event, (object) :: define o evento que deverá ser salvo. Pode ser button_press, key_press, etc.
                       No presente caso, tratar-se-á do button_press: botões do mouse.
    """
    click = "double" if event.dblclick else "single"
    msg = "{} click: button: {}; x = {}; y = {}; xdata = {}; ydata = {}".format(
            click, event.button, event.x, event.y, event.xdata, event.ydata)
    print(msg)

# Define o connectionid:
# fig.canvas.mpl_connect é um método que, quando lê uma evento (button press, por exemplo), chama uma função
# usando o evento lido como argumento.
cid = fig.canvas.mpl_connect("button_press_event", onclick)

single click: button: 1; x = 401; y = 304; xdata = 5.957056451612901; ydata = 0.5158801548001619
double click: button: 1; x = 401; y = 304; xdata = 5.957056451612901; ydata = 0.5158801548001619
single click: button: 1; x = 381; y = 314; xdata = 5.557862903225805; ydata = 0.5368840484904441
double click: button: 1; x = 377; y = 314; xdata = 5.478024193548386; ydata = 0.5368840484904441
single click: button: 1; x = 297; y = 234; xdata = 3.881249999999999; ydata = 0.3688528989681866
single click: button: 1; x = 285; y = 199; xdata = 3.6417338709677414; ydata = 0.295339271052199
single click: button: 1; x = 287; y = 149; xdata = 3.6816532258064503; ydata = 0.190319802600788
single click: button: 1; x = 247; y = 135; xdata = 2.883266129032257; ydata = 0.16091435143439292
single click: button: 1; x = 129; y = 179; xdata = 0.5280241935483869; ydata = 0.2533314836716346
single click: button: 1; x = 231; y = 230; xdata = 2.56391129032258; ydata = 0.36045134149207375
single click: button: 1; x =

Exemplo: Key press

In [3]:
fig, ax  = plt.subplots()
ax.plot(np.random.rand(10))

def onpress(event):
    """
    Esta função define o que acontecerá com o plot ao clicar neste. A título de um simples exemplo, o plot
    imprimirá as coordenadas no clique e se o clique foi de botão esquerdo ou direito.
    
    Entrada: 
    event, (object) :: define o evento que deverá ser salvo. Pode ser button_press, key_press, etc.
                       No presente caso, tratar-se-á do key_press_event: botões do teclado.
    """
    
    msg = "A tecla '{}' foi pressionada. As coordenadas salvas são x = {}, y = {}".format(
            event.key, event.x, event.y)
    
    print(msg)

# Define o connectionid
cid = fig.canvas.mpl_connect("key_press_event", onpress)

A tecla 'a' foi pressionada. As coordenadas salvas são x = 393, y = 282
A tecla 'c' foi pressionada. As coordenadas salvas são x = 393, y = 282
A tecla 'v' foi pressionada. As coordenadas salvas são x = 393, y = 282
A tecla 'b' foi pressionada. As coordenadas salvas são x = 393, y = 282
A tecla 'f' foi pressionada. As coordenadas salvas são x = 393, y = 282
A tecla 'f' foi pressionada. As coordenadas salvas são x = 1250, y = 572


## Exemplo: ligar os pontos

Obs: se arr = [1], então `element, = arr` implica `element = 1`; `element = arr` implica `element = [1]`. Consulte [isto aqui](https://stackoverflow.com/questions/65337288/line-ax-plotx-y).

`ax.plot()[0]` fornece um objeto `Line 2d`, cujos atributos e métodos podem ser consultados [aqui](https://matplotlib.org/stable/api/_as_gen/matplotlib.lines.Line2D.html).

`mpl_connect()` liga um evento a um canvas. 


+ [Plot specific Line2D](https://stackoverflow.com/questions/28688210/use-line2d-to-plot-line-in-python)

In [4]:
class LineBuilder:
    def __init__(self, line):
        
        # Redefine as coords para vazio. Desta forma, a figura poligonal terá início no primeiro ponto escolhido,
        # não na origem.
        line.set_data([],[])
        
        self.line = line
        self.xs = list(line.get_xdata()) # Recupera todos os x dos pontos em Line2D
        self.ys = list(line.get_ydata()) # Recupera todos os y dos pontos em Line2D
        self.cid = line.figure.canvas.mpl_connect('button_press_event', self) 
            # Este connectionid chama a instance do LineBuilder todas as vezes que há um "button_press_event".
            # chama: __call__.

    def __call__(self, event):
#         print('click', event)

        if event.inaxes!=self.line.axes: return 
            # Se o mouse click estiver fora do ax, nada ocorrerá; o __call__ termina.
            # Se o mouse click estiver dentro do ax, então novos pontos são adicionados ao gráfico.
        
        self.xs.append(event.xdata) 
        self.ys.append(event.ydata)
        self.line.set_data(self.xs, self.ys)
#         print(self.xs)
        
        # Renderiza o novo gráfico.
        # fig.canvas.draw() basicamente re-renderiza a figura.
        self.line.figure.canvas.draw()

fig, ax = plt.subplots()
ax.set(xlim=[-1,1], ylim=[-1,1], title="Click to build line segments")
line, = ax.plot([0],[0])  # Linha vazia; início do canvas.
linebuilder = LineBuilder(line)
plt.show()

## Draggable rectangle exercise
Write draggable rectangle class that is initialized with a Rectangle instance but will move its xy location when dragged. Hint: you will need to store the original xy location of the rectangle which is stored as rect.xy and connect to the press, motion and release mouse events. When the mouse is pressed, check to see if the click occurs over your rectangle (see Rectangle.contains) and if it does, store the rectangle xy and the location of the mouse click in data coords. In the motion event callback, compute the deltax and deltay of the mouse movement, and add those deltas to the origin of the rectangle you stored. The redraw the figure. On the button release event, just reset all the button press data you stored as None.

## Atenção:

Absolutamente toda a documentação está [aqui](https://matplotlib.org/stable/api/backend_bases_api.html), tudo em uma única página. Se tiver dúvida sobre um evento, recomendo fortemente usar

```python
...
def test_event_func(event):
    print(dir(event))

cid = fig.canvas.mpl_connect(event_name, test_event_func)
...
```

Isto imprimirá todos os atributos e todos os métodos do evento.

**Rectangle:**
+ [`Rectangle` documentation](https://matplotlib.org/stable/api/_as_gen/matplotlib.patches.Rectangle.html)
+ [`Contains` method](https://matplotlib.org/stable/api/_as_gen/matplotlib.artist.Artist.contains.html)

In [8]:
class DraggableRect():
    
    rect_params = {"xy":[0,0],'width':2,'height':4,"facecolor":'tab:blue'}
    ax_param = {'xlim':[-1,5],'ylim':[-1,5],'title':"Move the rectangle around"}
    rect = patches.Rectangle(**rect_params)

    fig,ax = plt.subplots()
    ax.add_patch(rect)
    ax.set(**ax_param)
    
    def __init__(self):
        self.cid1 = self.fig.canvas.mpl_connect("button_press_event",self.on_click)
        self.cid2 = self.fig.canvas.mpl_connect("motion_notify_event",self.on_motion)
        self.cid3 = self.fig.canvas.mpl_connect("button_release_event",self.on_release)
        self.mouse_x = None
        self.mouse_y = None
    
    def on_click(self,event):
        contains,_ = self.rect.contains(event)
#         print('Inside') if contains else print('Outside')
        if not contains: return
        self.mouse_x = event.xdata
        self.mouse_y = event.ydata
        
    
    def on_motion(self,event):
        # Todas essas verificações são necessárias. Se o mouse for para fora da região da figura, não há como mover
        # o retângulo para lá. Se o clique anterior não tiver sido no retângulo, não deverá o mover.
        if (self.mouse_x == None or self.mouse_y == None
            or event.xdata == None or event.ydata==None): return
        
        delta_x = event.xdata - self.mouse_x
        delta_y = event.ydata - self.mouse_y
        
        rect_x = self.rect.get_x()
        rect_y = self.rect.get_y()
        
        self.rect.set(xy=[rect_x+delta_x, rect_y+delta_y])
        self.fig.canvas.draw()
        
        self.mouse_x = event.xdata
        self.mouse_y = event.ydata
        
        
    
    def on_release(self,event):
#         print(self.mouse_xy)
        self.mouse_x = None
        self.mouse_y = None
                    
dr = DraggableRect()

Consegui. Depois de muito tempo consultando a documentação, consegui fazer isto sozinho.

## O teste que importa:

A figura é exibida. As coordenadas do clique são lidas e uma função principal é executada. Esta função principal chama uma função secundária.

In [146]:
fig,ax = plt.subplots()
ax.set(xlim=[-10,10], ylim=[-10,10])

def on_click(event):
    ax.clear()
#     print('Houve mouse click nas coordenadas a seguir:\n \
#           x = {}, y = {}'.format(event.xdata,event.ydata))
    func()

def func():
#     print("\n\n func() executada. Será plotado um gráfico aleatório.\n\n")
    ax.plot(np.random.rand(10))
    fig.canvas.draw()


cid = fig.canvas.mpl_connect("button_press_event", on_click)
fig.show()
